In [0]:
from rootpy.plotting import Hist, HistStack, Canvas, Legend
from ROOT import KinFtCM, RooArgSet
from ROOT import vector, RooRealVar, RooLognormal, RooProdPdf
from ROOT import RooFit, RooAddition, RooMinuit, RooArgList
import os
from numpy import sqrt
from rootpy.io import root_open
from rootpy.plotting import Hist, Canvas
from ROOT import TF1, TPaveText


tag_dict = { "CSVv2L": ["pfCombinedInclusiveSecondaryVertexV2BJetTags", 0.604 ],
             "CSVv2M": ["pfCombinedInclusiveSecondaryVertexV2BJetTags", 0.890 ],
             "CSVv2T": ["pfCombinedInclusiveSecondaryVertexV2BJetTags", 0.970 ]}

In [0]:

lumi = 1263.88
s_name = "CSVv2M"
option = "_second"
min_tag_cat = 200
tagger = tag_dict[s_name][0]
wp = tag_dict[s_name][1]
dir_path = "../scripts/13TeV_25ns/"
json_dir = "../output/13TeV_25ns/"


In [0]:
fit_status = "bfit"


data_names = ["SingleMuon_RRD_25ns", 
              "SingleMuon_PRD_25ns"]


mc_names = [["TTJets_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8_25ns",14170485.,831.76,0],
            ["WJetsToLNu_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8_25ns",16541248., 61526.7,1],
            ["DYJetsToLL_M-50_TuneCUETP8M1_13TeV-amcatnloFXFX-pythia8_25ns", 19310834., 6025.2, 1],
            ["ST_t-channel_4f_leptonDecays_13TeV-amcatnlo-pythia8_TuneCUETP8M1_25ns",6442497.,70.69,1],
            ["ST_tW_top_5f_inclusiveDecays_13TeV-powheg-pythia8_TuneCUETP8M1_25ns", 995600, 35.6,1],
            ["ST_tW_antitop_5f_inclusiveDecays_13TeV-powheg-pythia8_TuneCUETP8M1_25ns",1000000, 35.6, 1],
            ["QCD_Pt-1000toInf_MuEnrichedPt5_TuneCUETP8M1_13TeV_pythia8_25ns",1981954,10.4305*0.15544,1],
            ["QCD_Pt-50to80_MuEnrichedPt5_TuneCUETP8M1_13TeV_pythia8_25ns",5069469,19222500*0.02276,1],
            ["QCD_Pt-80to120_MuEnrichedPt5_TuneCUETP8M1_13TeV_pythia8_25ns",2926805,2758420*0.03844,1],
            ["QCD_Pt-120to170_MuEnrichedPt5_TuneCUETP8M1_13TeV_pythia8_25ns",4026104,469797*0.05362,1],
            ["QCD_Pt-170to300_MuEnrichedPt5_TuneCUETP8M1_13TeV_pythia8_25ns",3942640,117989*0.07335,1],
            ["QCD_Pt-300to470_MuEnrichedPt5_TuneCUETP8M1_13TeV_pythia8_25ns",3910268,7820.25*0.10196,1],
            ["QCD_Pt-470to600_MuEnrichedPt5_TuneCUETP8M1_13TeV_pythia8_25ns",1928421,645.528*0.12242,1],
            ["QCD_Pt-600to800_MuEnrichedPt5_TuneCUETP8M1_13TeV_pythia8_25ns",1983363,187.109*0.13412,1],
            ["QCD_Pt-800to1000_MuEnrichedPt5_TuneCUETP8M1_13TeV_pythia8_25ns",1982314,32.3486*0.14552,1]]

data_samples = []
for s in data_names:
    data_samples.append([json_dir+s+".json"])

mc_samples = []
for s in mc_names:
    mc_samples.append([json_dir+s[0]+".json",s[1],s[2],s[3]])

m = KinFtCM.Builder(tagger, wp, lumi)
for mc_s in mc_samples:
    m.add_mc_component(*mc_s)
for data_s in data_samples:
    m.add_data_component(*data_s)

m.zeroNegativeFracs = False
m.useDataPretagNorm = False

# min data counts per pretag category
cats = m.add_all_categories(True,min_tag_cat)
m.set_mc_jet_tag_effs()

data = m.get_data_hist()

In [0]:
for mc_s in mc_samples:
    print "{} - {} - {}".format(mc_s[0], mc_s[1], mc_s[2])

In [0]:
c = Canvas(1900,800)

h_format = {"data" : [["data"], None, "black", 0],
            "t#bar{t}": [[0],"solid","red", 0],
            "single top": [[3,4,5],"solid","yellow", 0],
            "Z+jets": [[2],"solid","orange", 0],
            "W+jets": [[1],"solid","blue",0],
            "QCD": [[6,7,8,9,10,11,12,13,14],"solid","green",0]}

bins = range(cats.size()+1)

h_dict = {}
h_stack = HistStack()
legend = Legend(4, leftmargin= 0.65)
legend.SetBorderSize(0)
legend.SetTextFont(43)
legend.SetTextSize(40)

for k,v in h_format.items():
    h_dict[k] = Hist( bins, title = k)
    for i, pretag_cat in enumerate(cats):
        h_dict[k].axis().SetBinLabel(i+1,pretag_cat)
        h_dict[k].SetXTitle("Jet Multiplicity per Kinematic Category")
        h_dict[k].SetYTitle("# events")
        data_counts = m.get_data_pretag_counts(pretag_cat)
        mcs_counts = m.get_mcs_pretag_counts(pretag_cat)
        if k == "data":
            h_dict[k][i+1] = (data_counts, sqrt(data_counts))
        else:
            mc_counts = 0.0
            for s_i in v[0]:
                mc_counts += mcs_counts[s_i]
            h_dict[k][i+1] = mc_counts
    h_dict[k].fillstyle = h_format[k][1]
    h_dict[k].fillcolor = h_format[k][2]
    h_dict[k].linecolor = h_format[k][2]
    h_dict[k].linewidth = h_format[k][3]
   


for s in ["QCD","single top", "Z+jets","W+jets", "t#bar{t}"]:
    h_stack.Add(h_dict[s])
    legend.AddEntry(h_dict[s], style = "F")

    
h_stack.Draw("HIST E1 X0")
h_stack.SetTitle("KinFtCM inclusive" + " - CMS preliminary - {}".format(lumi) + " pb^{-1} at #sqrt{s} = 13 TeV ")
h_stack.GetXaxis().LabelsOption("v")
h_stack.GetXaxis().SetTitle("selected jet categories")
h_stack.GetXaxis().SetTitleSize(0.045)
h_stack.GetXaxis().SetLabelSize(0.06)
h_stack.GetYaxis().SetTitle("# events")
h_stack.GetYaxis().SetLabelSize(0.05)

h_dict["data"].SetMarkerSize(2)
h_stack.SetMaximum(h_stack.GetMaximum()*1.1)
h_dict["data"].Draw("SAME E1")
legend.AddEntry(h_dict["data"], style='LEP')
legend.Draw()

name = "sel_jet_cat_13tev"
c.SaveAs(name+".pdf")
c.SaveAs(name+".root")
c


In [0]:


c = Canvas(3000,3000)

h_format = {"data" : [["data"], None, "black", 0],
            "t#bar{t}": [[0],"solid","red", 0],
            "single top": [[3,4,5],"solid","yellow", 0],
            "Z+jets": [[2],"solid","orange", 0],
            "W+jets": [[1],"solid","blue",0],
            "QCD": [[6,7,8,9,10,11,12,13,14],"solid","green",0]}


pretag_tag_cats = m.cat_set_
bins = range(pretag_tag_cats.size()+1)

h_dict = {}
h_stack = HistStack()
legend = Legend(3, leftmargin= 0.66)
legend.SetBorderSize(0)
legend.SetTextFont(43)
legend.SetTextSize(60)

for k,v in h_format.items():
    h_dict[k] = Hist( bins, title = k)
    for i, tag_cat in enumerate(pretag_tag_cats):
        h_dict[k].axis().SetBinLabel(i+1,tag_cat.first+":"+tag_cat.second)
        data_counts = m.get_data_tag_counts(tag_cat.first,tag_cat.second)
        mcs_counts = m.get_mcs_tag_counts(tag_cat.first,tag_cat.second)
        if k == "data":
            h_dict[k][i+1] = (data_counts, sqrt(data_counts))
        else:
            mc_counts = 0.0
            for s_i in v[0]:
                mc_counts += mcs_counts[s_i]
            h_dict[k][i+1] = mc_counts
    h_dict[k].fillstyle = h_format[k][1]
    h_dict[k].fillcolor = h_format[k][2]
    h_dict[k].linecolor = h_format[k][2]
    h_dict[k].linewidth = h_format[k][3]
   


for s in ["QCD","single top", "Z+jets","W+jets", "t#bar{t}"]:
    h_stack.Add(h_dict[s])
    legend.AddEntry(h_dict[s], style = "F")

h_stack.Draw("'HIST E1 X0")

h_stack.GetXaxis().LabelsOption("v")
h_stack.GetXaxis().SetLabelSize(0.030)
h_stack.SetMaximum(h_stack.GetMaximum()*1.3)
h_dict["data"].Draw("SAME E1")
legend.AddEntry(h_dict["data"], style='LEP')
if fit_status == "afit":
    h_stack.SetTitle("KinFtCM inclusive (after fit) - {}".format(s_name) + " - CMS preliminary - {}".format(lumi) + " pb^{-1} at #sqrt{s} = 13 TeV ")
else:
    h_stack.SetTitle("KinFtCM inclusive (before fit) - {}".format(s_name) + " - CMS preliminary - {}".format(lumi) + " pb^{-1} at #sqrt{s} = 13 TeV ")

    
h_stack.GetXaxis().SetTitle("selected:tag categories")
h_stack.GetYaxis().SetTitle("# events")
h_stack.GetYaxis().SetLabelSize(0.020)
h_stack.GetYaxis().SetTitleSize(0.025)
h_stack.GetYaxis().SetTitleOffset(1.8)
h_dict["data"].SetMarkerSize(2)



h_stack.GetXaxis().SetTitleSize(0.025)
h_stack.GetXaxis().SetTitleOffset(1.35)

legend.Draw()

c.SaveAs("tag_jet_cat_13tev_{}_{}{}.pdf".format(s_name, fit_status, option))
c.SaveAs("tag_jet_cat_13tev_{}_{}{}.root".format(s_name, fit_status, option))
c

In [0]:

fit_status = "afit"
root_file = root_open("{}fit_result_{}_{}{}.root".format(dir_path, tagger, wp, option))
fit_result = root_file.fitresult_sim_kin_pdf_data_hist
i_pars = fit_result.floatParsInit()
f_pars = fit_result.floatParsFinal()

for n_bin in range(m.b_jet_tag_effs_.getSize()):
    m.b_jet_tag_effs_[n_bin].setVal(f_pars[f_pars.index("b_tag_eff_{}".format(n_bin))].getVal())

m.xsecs_[0].setVal(f_pars[f_pars.index("xsec_"+mc_names[0][0])].getVal())
m.kappa_.setVal(f_pars[f_pars.index("kappa")].getVal())

In [0]:


b_eff_data = f_pars[f_pars.index("b_tag_eff_{}".format(n_bin))].getVal()
b_eff_data_err = f_pars[f_pars.index("b_tag_eff_{}".format(n_bin))].getError()
b_eff_mc = i_pars[f_pars.index("b_tag_eff_{}".format(n_bin))].getVal()
b_sf = (b_eff_data/b_eff_mc)
b_sf_err = (b_eff_data/b_eff_mc)*(b_eff_data_err/b_eff_data)
ttbar_xsec = f_pars[f_pars.index("xsec_"+mc_names[0][0])].getVal()
ttbar_xsec_err = f_pars[f_pars.index("xsec_"+mc_names[0][0])].getError()
kappa = f_pars[f_pars.index("kappa")].getVal()
kappa_err =  f_pars[f_pars.index("kappa")].getError()

print "{} &".format(s_name),
print "${:.0f}\pm{:.0f}$ &".format(ttbar_xsec, ttbar_xsec_err),
print "${:.2f}\pm{:.2f}$ &".format(kappa, kappa_err),
print "${:.2f}\pm{:.2f}$ &".format(b_eff_data, b_eff_data_err),
print "${:.2f}\pm{:.2f}$ \\\\".format(b_sf, b_sf_err),


In [0]:

c = Canvas(3000,3000)

h_format = {"data" : [["data"], None, "black", 0],
            "t#bar{t}": [[0],"solid","red", 0],
            "single top": [[3,4,5],"solid","yellow", 0],
            "Z+jets": [[2],"solid","orange", 0],
            "W+jets": [[1],"solid","blue",0],
            "QCD": [[6,7,8,9,10,11,12,13,14],"solid","green",0]}

pretag_tag_cats = m.cat_set_
bins = range(pretag_tag_cats.size()+1)

h_dict = {}
h_stack = HistStack()
legend = Legend(3, leftmargin= 0.66)
legend.SetBorderSize(0)
legend.SetTextFont(43)
legend.SetTextSize(60)

for k,v in h_format.items():
    h_dict[k] = Hist( bins, title = k)
    for i, tag_cat in enumerate(pretag_tag_cats):
        h_dict[k].axis().SetBinLabel(i+1,tag_cat.first+":"+tag_cat.second)
        data_counts = m.get_data_tag_counts(tag_cat.first,tag_cat.second)
        mcs_counts = m.get_mcs_tag_counts(tag_cat.first,tag_cat.second)
        if k == "data":
            h_dict[k][i+1] = (data_counts, sqrt(data_counts))
        else:
            mc_counts = 0.0
            for s_i in v[0]:
                mc_counts += mcs_counts[s_i]
            h_dict[k][i+1] = mc_counts
    h_dict[k].fillstyle = h_format[k][1]
    h_dict[k].fillcolor = h_format[k][2]
    h_dict[k].linecolor = h_format[k][2]
    h_dict[k].linewidth = h_format[k][3]
   


for s in ["QCD","single top", "Z+jets","W+jets", "t#bar{t}"]:
    h_stack.Add(h_dict[s])
    legend.AddEntry(h_dict[s], style = "F")

h_stack.Draw("'HIST E1 X0")

h_stack.GetXaxis().LabelsOption("v")
h_stack.GetXaxis().SetLabelSize(0.030)
h_stack.SetMaximum(h_stack.GetMaximum()*1.3)
h_dict["data"].Draw("SAME E1")
legend.AddEntry(h_dict["data"], style='LEP')
if fit_status == "afit":
    h_stack.SetTitle("KinFtCM inclusive (after fit) - {}".format(s_name) + " - CMS preliminary - {}".format(lumi) + " pb^{-1} at #sqrt{s} = 8 TeV ")
else:
    h_stack.SetTitle("KinFtCM inclusive (before fit) - {}".format(s_name) + " - CMS preliminary - {}".format(lumi) + " pb^{-1} at #sqrt{s} = 8 TeV ")

    
h_stack.GetXaxis().SetTitle("selected:tag categories")
h_stack.GetYaxis().SetTitle("# events")
h_stack.GetYaxis().SetLabelSize(0.020)
h_stack.GetYaxis().SetTitleSize(0.025)
h_stack.GetYaxis().SetTitleOffset(1.8)
h_dict["data"].SetMarkerSize(2)



h_stack.GetXaxis().SetTitleSize(0.025)
h_stack.GetXaxis().SetTitleOffset(1.35)

legend.Draw()

c.SaveAs("tag_jet_cat_13tev_{}_{}{}.pdf".format(s_name, fit_status, option))
c.SaveAs("tag_jet_cat_13tev_{}_{}{}.root".format(s_name, fit_status, option))
c